In [1]:
import pandas as pd 
import numpy as np 
from math import sqrt

In [2]:
moviedata = pd.read_csv('file:///Volumes/WORK/Data%20Science/practice%20data%20sets/Movies/movies.csv')
moviedata['Year'] = moviedata.title.str.extract('(\(\d\d\d\d\))', expand = False)
moviedata['Year'] = moviedata.Year.str.extract('(\d\d\d\d)', expand = False)

moviedata['title'] = moviedata.title.str.replace('(\(\d\d\d\d\))','')
moviedata['title'] = [x.strip() for x in moviedata.title]
moviedata = moviedata.drop('Year', axis = 1)

moviedata.head(100)


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy
...,...,...,...
95,97,"Hate (Haine, La)",Crime|Drama
96,98,Shopping,Action|Thriller
97,99,Heidi Fleiss: Hollywood Madam,Documentary
98,100,City Hall,Drama|Thriller


In [3]:
ratingsdata = pd.read_csv('file:///Volumes/WORK/Data%20Science/practice%20data%20sets/movies/ratings.csv')

ratingsdata = ratingsdata.drop('timestamp', axis = 1)

ratingsdata.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [4]:
def collaberative_recommender(movies , ratings):
    input_movies = pd.DataFrame({'title': movies , 'rating': ratings})
    input_id = moviedata[moviedata['title'].isin(input_movies['title'].to_list())]
    
    input_movies = pd.merge(input_id, input_movies)
    input_movies = input_movies.drop('genres', axis = 1)
    
    user_ratings_for_movies = ratingsdata[ratingsdata['movieId'].isin(input_movies['movieId'].to_list())]
    
    grouped_users = user_ratings_for_movies.groupby(['userId'])
    grouped_users = sorted(grouped_users, key = lambda x: len(x[1]), reverse = True)
    grouped_users = grouped_users[0:100]
    
    pearsoncorrelation_for_users = {}
    
    for name, group in grouped_users:
        group = group.sort_values(by = 'movieId')
        input_movies = input_movies.sort_values(by = 'movieId')
        nratings = len(group)
        
        our_usergroup_df_for_y_correl = input_movies[input_movies['movieId'].isin(group['movieId'])]
        our_usergroupratings_for_y_correl = our_usergroup_df_for_y_correl['rating'].to_list()
        
        previoususerratings_for_x_correl = group['rating'].to_list()
        
        Sxx = sum([i**2 for i in previoususerratings_for_x_correl]) - pow(sum(previoususerratings_for_x_correl),2)/float(nratings)
        Syy = sum([i**2 for i in our_usergroupratings_for_y_correl]) - pow(sum(our_usergroupratings_for_y_correl),2)/float(nratings)
        Sxy = sum([a*b for a,b in zip(previoususerratings_for_x_correl,our_usergroupratings_for_y_correl)]) - sum(previoususerratings_for_x_correl) * sum(our_usergroupratings_for_y_correl)/nratings
        
        if Sxx == 0 or Syy == 0:
            pearsoncorrelation_for_users[name] = 0
        else:
            pearsoncorrelation_for_users[name] = Sxy/sqrt(Sxx*Syy)

    pearsondf = pd.DataFrame.from_dict(pearsoncorrelation_for_users , orient = 'index')
    pearsondf.columns = ['Similarity Index']
    pearsondf['userId'] = pearsondf.index
    pearsondf.index = range(len(pearsondf))
    
    topusers = pearsondf.sort_values(by = 'Similarity Index', ascending = False)
    topusers_30 = topusers[0:30]
    topusers_30_ratings = topusers_30.merge(ratingsdata, left_on = 'userId', right_on = 'userId', how = 'inner')
    topusers_30_ratings['weighted rating'] = topusers_30_ratings['rating']*topusers_30_ratings['Similarity Index']
    topusers_30_ratings.head()
    
    weighted_rating_sum_of_each_movie = topusers_30_ratings.groupby('movieId').sum()[['Similarity Index', 'weighted rating']]
    weighted_rating_sum_of_each_movie.columns = ['Similarity Index Total', 'sum weighted rating']
    weighted_rating_sum_of_each_movie['Movie Score'] = weighted_rating_sum_of_each_movie['sum weighted rating']/weighted_rating_sum_of_each_movie['Similarity Index Total']
    weighted_rating_sum_of_each_movie['movieId'] =  weighted_rating_sum_of_each_movie.index
    weighted_rating_sum_of_each_movie =  weighted_rating_sum_of_each_movie.sort_values(by = 'Movie Score', ascending = False)

    movies_to_recommend = weighted_rating_sum_of_each_movie[:5]
    movies_to_recommend = moviedata[moviedata['movieId'].isin(movies_to_recommend['movieId'])]
    movies_to_recommend.index = range(1,len(movies_to_recommend.title)+1)
    movies_to_recommend = movies_to_recommend.title.to_frame()
    
    return movies_to_recommend


        

In [38]:
movies =  ['Star Wars: Episode IV - A New Hope',
'Indiana Jones and the Last Crusade' ,'Robot Chicken: Star Wars']
ratings = [4,5,5]

collaberative_recommender(movies, ratings)

,title
1,Romper Stomper
2,Rudy
3,Rosetta
4,Live from Baghdad
5,Clerks II


In [39]:
list1 = list(moviedata.title.unique())
list1

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'American President, The',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'City of Lost Children, The (Cité des enfants perdus, La)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao)',
 'Dangerous Minds',
 'Twelve Monkeys (a.k.a. 12 Monkeys)',
 'Wings of Courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry, the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How to Make an American Quilt',
 'Seven (a.k.a. Se7en)',
 'Pocahontas',
 'When Night Is Falling',
 

In [45]:
for x in list1:
    if 'Sport' in x:
        print(x)

Man's Favorite Sport?
This Sporting Life
Schooled: The Price of College Sports
Sport, Sport, Sport
Bar Sport
Sportloto-82
You're a Good Sport, Charlie Brown
